# Introduction
This tutorial covers how to get started with your own Julia package. It assumes some familiarity with the language and that you have a working version of Julia installed on your computer. We also highly recommend installing the [Juno IDE](http://junolab.org/).

This tutorial is built for Julia v1.0.0+.

# Getting Started

## Getting Packages
Julia v1.0 has a brand-new package manager with a lot of 
functionality. We'll cover some of the basics here.

### Using the Package Manager
To use the package manager from the REPL (recommended) press the `]` key.

### File Locations
If you installed Julia with default settings, it should place all files beneath your home directory:

`/home/username/.julia`

A couple of important sub-directories:
* `dev` - contains git repositories of the packages being developed. It is okay to edit files in this directory
* `packages` - contains git repositories of the packages downloaded via the package manager. You should not edit these files.
* `compiled` - contains compiled files (`*.ji`) for pre-compiled packages.


### Adding a registered package
To add a registered package use the `add` command within the Package Manager. This will install the package on your computer and add it to your active project. For example, to add the ForwardDiff package simple enter

`(v1.0) pkg> add ForwardDiff`


### Using a Package
Once installed, a package is brought into the the workspace using the `using` command:


In [10]:
using ForwardDiff

## Creating your Own Package
1. Go to GitHub.com and create a new repository. By convention it should end with `*.jl` (i.e. `NewPackage.jl`).
![Add_GitHub_Repo](../img/AddRepo.png)

2. Add new repository to developed Julia packages using the Package Manager

```
(v1.0) pkg> dev https://github.com/username/NewPackage.jl.git
```

3. Navigate to repository. 

```shell
cd /home/user/.julia/dev/NewPackage
```

4. Create `src` sub-directory. This holds the core project code.

```shell
mkdir src
```

5. Create the main project file within your source folder using your favorite editor. This needs to have the same name as the Package, e.g. `src\NewPackage.jl`. To do this from Juno, select File|Open Folder and open the repository. Open a new file (CTRL-N) and save it to the `src` folder. The file needs to have the following contents:


In [7]:
# <src/NewPackage.jl 
module NewPackage
    greet() = "Hi there, welcome to NewPackage!"
end

Main.NewPackage

## Using your Package
### REPL
To use your new package from the REPL simply enter 
```julia
using NewPackage
NewPackage.greet()
```

### Juno
The process is identical, simply enter the previous commands into the Julia Console. 
To open the console go to Julia|Open Console, enter (CTRL+J,CTRL+O), or `Julia Client: Open Console` from the Command Pallette (CTRL+SHIFT+P)

# Using Different Files
Typically the main project file does not contain source code; instead, it collects code from other files and takes care of exporting critical functions. We'll cover all of that here. First, we will cover the `include` function.

## `Include` function
The `include` function essentially copies the code from a source file into another. Let's say I have a file with the following contents:

```julia
# <test_include.jl>
myfun(x) = x + 2 + e^x
a = 4
b = c + 2;
```

If we now use `include("test_include.jl")` it will run the contents of the file:

In [1]:
include("test_include.jl")

LoadError: UndefVarError: c not defined

Notice we get an error because the variable `c` is not defined. If we define `c` and call include again we get the expected behavior

In [10]:
c = 5
include("test_include.jl")
@show a
@show b
@show myfun(2);

a = 4
b = 7
myfun(2) = 8


In practice, we use this to "include" files into the main project file. Typically, these file do not contain anything except functions (and possibly sub-modules). For example, let's say we want to add a new function to our project, located in `project_funtions.jl`

In [13]:
# <src/project_functions.jl>
newfun(x::Int) = x + 3
newfun(x::Float64) = x + π
newfun(x::String) = x * "π";

We now include this in our main project file

In [1]:
# <src/NewProject.jl>
module NewProject
    include("project_functions.jl")
    greet() = "Hi there, welcome to NewPackage!"
end

Main.NewProject

These functions are now part of our package. Test them out by opening up a **new** REPL (or restarting it inside Juno) and enter 
```julia
using NewPackage
```

In [3]:
# Note the use of multiple dispatch!
@show NewProject.newfun(1)
@show NewProject.newfun(1.0)
@show NewProject.newfun("1");

NewProject.newfun(1) = 4
NewProject.newfun(1.0) = 4.141592653589793
NewProject.newfun("1") = "1π"


## Exporting Functions
Note that we have to, rather inconveniently, prepend "MyPackage" to each of our function calls. To avoid this, we can export them. Exporting a function will allow the function to be used by itself in whatever scope into which it is imported. Note that we cannot replicate this same behavior here in a Jupyter notebook since we aren't actually manipulating packages here, simply creating modules.

In [4]:
# <src/NewProject.jl>
module NewProject
    export
        newfun,
        greet
    
    include("project_functions.jl")
    greet() = "Hi there, welcome to NewPackage!"
end

Main.NewProject

# Workflow

You may have already noticed that we requested you to reboot the REPL after each time you changed your package. This is necessary due to the way Julia handles packages within the global scope of the REPL. There are two ways of getting around this development headache:

1. Revise.jl
2. Juno IDE

### Revise.jl
Revise.jl is made specifically to get around this issue of having the reload the workspace when changes are made to a package/module. Basically you run it prior to loading in any other modules and it takes of most of the headache. 

### Juno IDE (recommended)
Juno has some clever ways to get around this issue, and has some other nice features that make your life as a developer easier. Here we will go over basic workflow to minimize how many times you need to restart the REPL.

1. Create a "working" script that acts basically like your REPL. You easily turn these scripts into test scripts later on.

2. Change the "working module" of the script by clicking "Main" in the status bar and enter the name of your package, or use the shortcut (CTRL+J CTRL+M). The status bar should change to display the name of you module once you do this. Note that this is file-dependent. You should notice that Juno does this automatically for files that are included in your main file. You can even do this for the REPL to access the variables in the workspace of your module.
![status bar](../img/Juno_status.png)
![change_module](../img/ChangeModule.png)

3. With the active module set to your package, any variables, functions, types, etc. that you create in your scipt will be created in the workspace of the your package, and has direct access to all variables and functions of your package (i.e. you don't need to use NewPackage.newfun, even if you haven't exported it). 

4. If you add or change a function in one of the files included in your project, you can reload it by hitting CTRL-RETURN (moves to next statement or function) or SHIFT-RETURN (doesn't move cursor). You can now re-run that function in your "working" script and will see the change.

5. DISCLAIMER: This will save you a lot of time, but it often a good idea to restart your REPL if you begin to observe odd behavior. It's not uncommon to have a variable, function, or method that you don't realize is there that affects behavior. When in doubt: restart the REPL. NOTE: You will ALWAYS needs to restart the REPL after making changes to custom types. 

### Example
Let's do this for our `newfun` function.
* Restart the REPL and enter `using NewPackage`. Open a new file (CTRL-N) named `newfun_test.jl`:
```julia
# <src/newfun_test.jl
newfun("1")
```

* Change the active workspace of `newfun_test.jl` to `NewPackage` using CTRL-J CTRL-M.

* With your cursor on the line testing `newfun`, hit CTRL-RETURN and you'll see the result pop up beside it:
![newfun_test](../img/newfun_test.png)

* Now open `src/project_functions.jl` in Juno and change the string operation to include a space. Afterwards hit CTRL-RETURN to update the function. You should the name of the function pop up beside it.
![newfun_change](../img/newfun_change.png)

* Return to `newfun_test.jl` and rerun the test using CTRL-RETURN. You should see the change reflected, without having to restart the REPL.
![newfun_updated](../img/newfun_updated.png)

# Testing

You should include unit tests as you develop your package. Include all tests in `test/runtests.jl`.
See [documentation](https://docs.julialang.org/en/v1/stdlib/Test/index.html) for details on unit testing. We give a brief example below. 


### Running Tests
You can run all tests for your package from the Package Manager:
```
(v1.0) pkg> test NewPackage
```
Or by calling the function directly from the package:
```julia
include("test/runtests.jl")
```
Or by opening the file in Juno and hitting CTRL-SHIFT-RETURN

You should always run these tests in the Main workspace, to replicate the actual testing environment. However, for debugging it is often useful to have the test suites in separate files that are included in `test/runtests.jl` via `include`. You change the workspace of these files to debug and modify your code while you work on it. 

### Travis CI
[Travis CI](https://travis-ci.org/) is a tool for automatically testing your package. A badge indicating the status of your build is usually displayed in your README.md on GitHub, and lends credibility to your project. Setting up a project with Travis CI is straight-forward, got to https://travis-ci.org/ and sign up with your GitHub account. Simply follow the prompts to set up Travis CI with your repository. You'll then need to create a `.travis.yml` file in your root directory. Here is an example that also sends results to some code coverage utilities:
```yml
# <.travis.yml>
## Documentation: http://docs.travis-ci.com/user/languages/julia/
language: julia
os:
  - linux
  - osx
julia:
  - 0.7
  # - nightly
notifications:
  email: false
git:
  depth: 99999999

## uncomment the following lines to allow failures on nightly julia
## (tests will run but not make your overall status red)
#matrix:
#  allow_failures:
#  - julia: nightly

## uncomment and modify the following lines to manually install system packages
addons:
 apt: # apt-get for linux
   packages:
   - gfortran
#before_script: # homebrew for mac
#  - if [ $TRAVIS_OS_NAME = osx ]; then brew install gcc; fi

## uncomment the following lines to override the default test script
#script:
#  - julia -e 'Pkg.clone(pwd()); Pkg.build("TrajectoryOptimization"); Pkg.test("TrajectoryOptimization"; coverage=true)'
after_success:
  # push coverage results to Coveralls
  - julia -e 'cd(Pkg.dir("TrajectoryOptimization")); Pkg.add("Coverage"); using Coverage; Coveralls.submit(Coveralls.process_folder())'
  # push coverage results to Codecov
  - julia -e 'cd(Pkg.dir("TrajectoryOptimization")); Pkg.add("Coverage"); using Coverage; Codecov.submit(Codecov.process_folder())'
```

### Example
* Create a new file `test\runtests.jl`

```julia
# <test/runtests.jl>
using Test
using JuliaIntro
@testset "newfun" begin
    @test newfun(1) == 4
    @test newfun(2.0) == 2+π
    @test newfun("a") == "a π"
end
```

* The @testset macro defines a set suite that is useful for grouping tests together.
* The @test macro tests for a boolean. See documentation for more options